# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124460e+02     1.762744e+00
 * time: 0.06020808219909668
     1     1.102562e+01     9.531346e-01
 * time: 1.2860229015350342
     2    -1.221848e+01     1.059624e+00
 * time: 1.3732779026031494
     3    -3.407235e+01     8.098673e-01
 * time: 1.4513189792633057
     4    -4.773417e+01     5.929650e-01
 * time: 1.5375230312347412
     5    -5.698632e+01     2.147011e-01
 * time: 1.6234588623046875
     6    -5.981623e+01     1.406836e-01
 * time: 1.6784799098968506
     7    -6.085580e+01     5.319556e-02
 * time: 1.7346270084381104
     8    -6.126281e+01     9.148275e-02
 * time: 1.8011069297790527
     9    -6.160678e+01     3.205312e-02
 * time: 1.8567900657653809
    10    -6.182963e+01     2.923432e-02
 * time: 1.9152560234069824
    11    -6.199046e+01     2.420956e-02
 * time: 1.9724700450897217
    12    -6.205445e+01     1.869328e-02
 * time: 2.03898286819458
    13    -6.212002e+01     1.354937e-02
 * time: 2.09697699546

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671062
    AtomicLocal         -18.8557682
    AtomicNonlocal      14.8522654
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485373 
    Xc                  -19.3336821

    total               -62.261666460738
